## Key concepts

### Relationship between macro- and micro-scale

The lack of pure homogeneity at a lower scale has a huge influence on the behaviour at the large scale. Small defects are known to often be the cause of catastrophic macroscopic failure. Macro-scale properties often depend on micro-scale features encapsulated in what is named structure-property relationship. Even further, some macro-scale behaviour can emerge exclusively from the micro-scale (example).

It is computationally impossible however to model those microscopic features when solving a macroscopic simulation as it requires a mesh resolution way too large to bridge the length scale separation (think about a TV needing to resolve every strand of hair to display a character’s face!).

Separation of scales occurs when the macroscopic length scale is larger than microscopic length scale by 3 orders of magnitude. Otherwise, concurrent/embedded multiscale method also exist (explain in further details) but we will focus on the first kind, hierarchical. Because the scales are separated, the micromodel and macromodel need to be solved separately and coupled with a multiscale method. They are appropriately solved according to the scale considered. For example, while the microstructure is discretised at the micro-scale, the macro-scale is solved as a continuum.

```{figure} ./figures/44296_2024_21_Fig3_HTML.jpg
---
height: 200px
name: sequential_vs_concurrent
---
Sequential vs concurrent multiscale modeling, applied to the fracturing problem of fiber-reinforced concrete. Figure from https://doi.org/10.1038/s44296-024-00021-z
```

```{figure} ./figures/Continuum-macrostructure-and-heterogeneous-microstructure-associated-with-point-P-Fig-2_W640.jpg
---
height: 200px
name: macro_vs_micro
---
Continuum macrostructure and heterogeneous microstructure associated with point P. Figure from https://doi.org/10.1007/s004660000212
```

To percolate information of the micro-scale to the macro-scale, we perform an upscaling using a homogenization method. The upscaled macroscopic response is called effective. Those multiscale methods are meant to couple the micro-scale and macro-scale models. In this first part we focus on the upscaling, which is a one-way coupling, upwards.

### Categories of multiscale methods

Different types of methods can be used to upscale. We can resort for example to simple empirical relationships which are the laws of mixtures. The first one from Voigt (1884) assumes that the effective medium is under a constant strain. For a two-phase medium, the effective value of the property is equivalent to setting two springs in parallel, and is expressed as:

$$
X_{eff}=\phi_1X_1+\left(1-\phi_1\right)X_2
$$

Where $\phi_1$ is the volume fraction of phase 1.
For Reuss law, the assumption is constant force and similarly the effective value of the property for a two-phase medium is equivalent to two springs in series and is expressed as

$$
\frac{1}{X_{eff}}=\frac{\phi_1}{X_1}+\frac{1-\phi_1}{X_2}
$$

Those two formulas correspond respectively to kinematic or traction condition. Kinematic condition pushes the system to its maximal entropy and corresponds therefore to an upper bound of the effective property estimation, while the traction condition is oppositely the minimum.

In [ ]:
#make a plot of the rule of mixtures like in wikipedia (https://en.wikipedia.org/wiki/Rule_of_mixtures)
import numpy as np

When only interested in predicting failure, we can employ limit analysis which can consider more details of the microstructure. It also finds either the upper or lower bound of the solution. When trying to consider the exact microstructure, we need to resort instead to the computational methods. In this computational modelling book, we focus on that last option.

### Representative Elementary Volume

While the micro-scale is heterogeneous whether in structure or in property distribution, the application of multiscale methods can only be valid when the large scale is statistically homogeneous. We define as such a Representative Elementary Volume (REV) above which size the effective property of interest does not fluctuate. The REV is defined for the property to upscale and the length scale considered. When focusing on upscaling the microstructure, the length scale is the characteristic length of voids, inclusions, fibers or crystals depending on the material. For porous rocks, it is the average grain size. And studies show that the REV size for hydraulic conductivity of a rock is often smaller than for the mechanical stiffness.

A sample stops being statistically homogeneous when encountering a high length scale like fractures for example. Only unfractured samples can be homogenized. But a fracture network can yield an effective response considering the fracture as the length scale of interest.

When the structure is periodic such as for artificial materials like 3D printed metamaterials, the REV is directly the unit cell and the upscaling is done using periodic conditions. In the case of natural non-periodic REV, the REV size needs to be sufficiently large enough to eliminate boundary effects.

```{figure} ./figures/Continuum-macrostructure-and-heterogeneous-microstructure-associated-with-point-P-Fig-2_W640.jpg
---
height: 200px
name: REV_vs_unit_cell
---
Representative Elementary Volume for a periodic composite material (left) and for a random material (right). Figure from https://doi.org/10.1142/S1756973711000509
```

### Averaging theorem
Effective quantities are obtained by averaging the microscopic fields. The average of the quantity $X$ over the domain omega is defined as:

$$
\left\langle X\right\rangle=\overline{X}=\frac{1}{V_\mathrm{\Omega}}\int_{\mathrm{\Omega}} X dV
$$

For the relationship macro to micro, we obtain:

$$
X_M=\overline{X_m}
$$

This relation is derived from the Hill-Mandel condition which expresses an irrefutable principle of thermodynamics. Hill’s lemma states that the energy has to be conserved across scales. As a translation, the macroscopic work is equal to the average of the microscopic work. From this condition, the averaging of other quantities can be derived similarly.

At the microscopic scale, any quantity can be expressed as a fluctuating field around the averaged value:

$$
X_m=\overline{X_m}+\widetilde{X_m}
$$

Where $\widetilde{X}$ is the fluctuation. Therefore we have:

$$
X_M=\overline{\overline{X_m}+\widetilde{X_m}}=\overline{X_m}+\overline{\widetilde{X_m}}
$$

This expression implies that the fluctuations need to average out on the domain for the averaging relation to be valid. This is indeed the case when considering the domain size above REV, as per its definition. It showcases once more the need to consider a domain above the REV size for upscaling.

### Upscaling
To obtain an effective property, we use the macroscopic constitutive law. The law is inverted  to compute the property using the macroscopic fields values obtained from our simulation. The simulation often considers boundary conditions of deformation or force. Those boundary conditions corresponds to imposing the macroscopic value of the variable. The secondary variable is obtained by homogenizing the microscopic field heterogeneously distributed in the simulation result.

Let us run through the example of upscaling the Young’s modulus of a porous material:

Set up a numerical experiment of uniaxial mechanical test with displacement-control of a sample of the porous material above the REV size. The simulation solves for elasticity at the micro-scale. The macroscopic constitutive law for elasticity -- Hooke’s law -- reads as:

$$
\sigma=E\cdot\epsilon
$$

The macroscopic axial strain can be directly calculated from the prescribed displacement boundary and the macroscopic axial stress is averaged from the microscopic axial stress over the full sample.
The effective Young’s modulus is calculated from the equation above.